Pixeltable's Microsoft Fabric integration enables you to access Azure OpenAI models within Microsoft Fabric notebook environments with automatic authentication.

## Prerequisites

- A Microsoft Fabric workspace with access to AI services
- Running in a Microsoft Fabric notebook environment

## Important notes

- This integration only works within Microsoft Fabric notebook environments
- Authentication is handled automatically - no API keys required
- Azure OpenAI usage in Fabric is subject to your organization's Fabric capacity and policies

For more information about Fabric AI services, see the [Microsoft Fabric AI Services documentation](https://learn.microsoft.com/en-us/fabric/data-science/ai-services/ai-services-overview).

First, install Pixeltable in your Fabric notebook:

In [ ]:
%pip install -qU pixeltable

Now let's create a Pixeltable directory to hold the tables for our demo.

In [ ]:
import pixeltable as pxt

# Remove the 'fabric_demo' directory and its contents, if it exists
pxt.drop_dir('fabric_demo', force=True)
pxt.create_dir('fabric_demo')

## Chat Completions with Standard Models

Let's start by using a standard chat model (gpt-4.1) for a simple Q&A application.

Create a table in Pixeltable with a computed column that calls Azure OpenAI via Fabric:

In [ ]:
from pixeltable.functions import fabric

# Create a table for customer support tickets
tickets = pxt.create_table('fabric_demo.support_tickets', {
    'ticket_id': pxt.Int,
    'customer_message': pxt.String,
    'priority': pxt.String
})

# Add a computed column that automatically generates AI responses
# No API keys needed - Fabric handles authentication!
messages = [
    {'role': 'system', 'content': 'You are a helpful customer support agent. Be concise and professional.'},
    {'role': 'user', 'content': tickets.customer_message}
]

tickets.add_computed_column(
    ai_response=fabric.chat_completions(
        messages,
        model='gpt-4.1',
        model_kwargs={
            'max_tokens': 200,
            'temperature': 0.7
        }
    )
)

In [ ]:
# Parse the response to extract just the message content
tickets.add_computed_column(
    response_text=tickets.ai_response.choices[0].message.content
)

In [ ]:
# Insert data - AI responses are generated automatically
tickets.insert([
    {'ticket_id': 1, 'customer_message': 'How do I reset my password?', 'priority': 'low'},
    {'ticket_id': 2, 'customer_message': 'My order hasn\'t arrived after 2 weeks', 'priority': 'high'},
    {'ticket_id': 3, 'customer_message': 'Can I change my subscription plan?', 'priority': 'medium'}
])

# Query results with AI-generated responses
tickets.select(
    tickets.ticket_id,
    tickets.customer_message,
    tickets.response_text
).show()

## Chat Completions with Reasoning Models

Fabric also supports reasoning models like gpt-5, which are optimized for complex reasoning tasks.

**Note:** Reasoning models have different parameter requirements:
- Use `max_completion_tokens` instead of `max_tokens`
- Don't support the `temperature` parameter

In [ ]:
# Create a table for complex reasoning tasks
reasoning_tasks = pxt.create_table('fabric_demo.reasoning', {
    'task_id': pxt.Int,
    'problem': pxt.String
})

messages = [
    {'role': 'user', 'content': reasoning_tasks.problem}
]

reasoning_tasks.add_computed_column(
    reasoning_output=fabric.chat_completions(
        messages,
        model='gpt-5',  # Reasoning model
        model_kwargs={
            'max_completion_tokens': 1000  # Note: max_completion_tokens, not max_tokens
        }
    )
)

reasoning_tasks.add_computed_column(
    solution=reasoning_tasks.reasoning_output.choices[0].message.content
)

In [ ]:
# Insert a complex reasoning task
reasoning_tasks.insert([
    {
        'task_id': 1,
        'problem': 'Explain how to implement a binary search tree with self-balancing capabilities. Include time complexity analysis.'
    }
])

reasoning_tasks.select(reasoning_tasks.problem, reasoning_tasks.solution).show()

## Embeddings for Semantic Search

Fabric also supports embedding models for semantic search and similarity operations.

Let's create a knowledge base with semantic search capabilities:

In [ ]:
# Create a knowledge base table
knowledge_base = pxt.create_table('fabric_demo.knowledge', {
    'doc_id': pxt.Int,
    'content': pxt.String,
    'category': pxt.String
})

# Add embeddings column
knowledge_base.add_computed_column(
    embedding=fabric.embeddings(
        knowledge_base.content,
        model='text-embedding-ada-002'
    )
)

# Insert some documents
knowledge_base.insert([
    {
        'doc_id': 1,
        'content': 'Pixeltable is a Python library for AI data workflows with built-in versioning.',
        'category': 'product'
    },
    {
        'doc_id': 2,
        'content': 'Microsoft Fabric provides a unified analytics platform for data engineering and AI.',
        'category': 'platform'
    },
    {
        'doc_id': 3,
        'content': 'Azure OpenAI Service offers powerful language models through REST APIs.',
        'category': 'service'
    }
])

In [ ]:
# Add an embedding index for fast similarity search
knowledge_base.add_embedding_index('content', embedding=fabric.embeddings.using(model='text-embedding-ada-002'))

In [ ]:
# Perform similarity search
sim = knowledge_base.content.similarity('AI platform for data science')
knowledge_base.select(knowledge_base.content, knowledge_base.category, sim=sim).order_by(sim, asc=False).limit(2).show()

## Combining Chat and Embeddings: RAG Pattern

Let's combine embeddings and chat completions to build a simple Retrieval-Augmented Generation (RAG) system:

In [ ]:
# Create a table for questions
questions = pxt.create_table('fabric_demo.questions', {
    'question_id': pxt.Int,
    'question': pxt.String
})

# Find similar documents using similarity search
@pxt.query
def retrieve_context(question: str, top_k: int = 2) -> list[dict]:
    sim = knowledge_base.content.similarity(question)
    return (
        knowledge_base
        .select(knowledge_base.content)
        .order_by(sim, asc=False)
        .limit(top_k)
        .collect()['content']
    )

# Add context retrieval
questions.add_computed_column(
    context=retrieve_context(questions.question, top_k=2)
)

# Build RAG prompt with retrieved context
questions.add_computed_column(
    rag_messages=[
        {
            'role': 'system',
            'content': "Answer the question based on the provided context. If the context doesn't contain relevant information, say so."
        },
        {
            'role': 'user',
            'content': f'Context: {questions.context}\n\nQuestion: {questions.question}'
        }
    ]
)

# Generate answer using gpt-4.1
questions.add_computed_column(
    answer_response=fabric.chat_completions(
        questions.rag_messages,
        model='gpt-4.1',
        model_kwargs={'max_tokens': 300}
    )
)

questions.add_computed_column(
    answer=questions.answer_response.choices[0].message.content
)

In [ ]:
# Ask a question
questions.insert([
    {'question_id': 1, 'question': 'What is Microsoft Fabric used for?'}
])

questions.select(
    questions.question,
    questions.context,
    questions.answer
).show()

## Available Models in Fabric

The following models are currently available in Microsoft Fabric:

**Chat Models:**
- `gpt-5` (reasoning model)
- `gpt-4.1`
- `gpt-4.1-mini`

**Embedding Models:**
- `text-embedding-ada-002`
- `text-embedding-3-small`
- `text-embedding-3-large`

For the latest information on available models, see the [Fabric AI Services documentation](https://learn.microsoft.com/en-us/fabric/data-science/ai-services/ai-services-overview).

## Key Features

- **Automatic Authentication**: No API keys required - authentication is handled by Fabric
- **Rate Limiting**: Pixeltable automatically handles rate limiting based on Azure OpenAI response headers
- **Batching**: Embedding requests are automatically batched for efficiency (up to 32 inputs per request)
- **Incremental Processing**: Computed columns only run on new or updated data
- **Versioning**: All data and transformations are automatically versioned

### Learn More

To learn more about advanced techniques in Pixeltable:
- [RAG Operations in Pixeltable](https://docs.pixeltable.com/howto/use-cases/rag-operations)
- [Working with Embeddings](https://docs.pixeltable.com/platform/embedding-indexes)
- [Microsoft Fabric AI Services](https://learn.microsoft.com/en-us/fabric/data-science/ai-services/ai-services-overview)

If you have any questions, don't hesitate to reach out on our [Discord community](https://discord.gg/QPyqFYx2UN).